In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 5]

In [ ]:
! rm -rf COVID-19 && git clone https://github.com/pcm-dpc/COVID-19

In [ ]:
import os
import pandas as pd

In [ ]:
original_df = pd.read_csv(os.path.join("COVID-19", "dati-province", "dpc-covid19-ita-province.csv"))

In [ ]:
original_df['data'] = pd.to_datetime(original_df['data'])
df = original_df[['data', 'codice_provincia', 'totale_casi', 'sigla_provincia']]
df = df.dropna()
df = df.loc[df['sigla_provincia'].isin(['BA', 'LE', 'TA', 'BT', 'FG'])]

In [ ]:
df = df.pivot(index="data", columns="sigla_provincia", values="totale_casi")

# Sampling original data
df = df.iloc[::10, :]
selected_dates = df.index

In [ ]:
plot = df.plot(linewidth=2.0, grid=True, title="Casi di COVID 19 in Puglia", xticks=df.index)
plot.set_xlabel("Data")
plot.set_ylabel("Casi")

fig = plot.get_figure()
fig.savefig("covid-cases-puglia.png")

In [ ]:
plot = df.diff().plot(linewidth=2.0, grid=True, title="Aumento Casi di COVID 19 in Puglia", xticks=df.index)
plot.set_xlabel("Data")
plot.set_ylabel("Crescita")
fig = plot.get_figure()
fig.savefig("covid-diff-cases-puglia.png")

In [ ]:
original_df = pd.read_csv(os.path.join("COVID-19", "dati-province", "dpc-covid19-ita-province.csv"))
df = original_df[['data', 'denominazione_regione', 'totale_casi']]
df = df.groupby(['data', 'denominazione_regione']).sum().reset_index()
df = df.pivot(index="data", columns="denominazione_regione", values="totale_casi")
plot = df.plot(linewidth=2.0, grid=True, title="Casi di COVID 19 in Italia per regione")
plot.set_xlabel("Data")
plot.set_ylabel("Casi")
plot.legend(bbox_to_anchor=(1, 1))

fig = plot.get_figure()
fig.savefig("covid-cases-italia.png")

In [ ]:
plot = df.diff().plot(linewidth=2.0, grid=True, title="Aumento casi di COVID 19 in Italia per regione")
plot.set_xlabel("Data")
plot.set_ylabel("Casi")
plot.legend(bbox_to_anchor=(1, 1))

fig = plot.get_figure()
fig.savefig("covid-cases-diff-italia.png")

In [ ]:
import folium 
def generateBaseMap(default_location=[41.89193, 12.51133], default_zoom_start=5):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

base_map = generateBaseMap()


In [ ]:


from folium.plugins import HeatMap
import folium.plugins as plugins
import random

original_df['data'] = pd.to_datetime(original_df['data'])


lat_long_df = original_df[['totale_casi', 'lat', 'long', 'data', 'sigla_provincia']]
lat_long_df = lat_long_df.dropna()

lat_long_df = lat_long_df.loc[lat_long_df['data'].isin(selected_dates)]
lat_long_df['weight'] = (lat_long_df['totale_casi'] - lat_long_df['totale_casi'].min()) / (lat_long_df['totale_casi'].max() - lat_long_df['totale_casi'].min())
lat_long_df['hm_data'] = lat_long_df.apply(lambda row: [row['lat'], row['long'], row['weight']],axis=1)
lat_long_df['weight'] = (lat_long_df['totale_casi'] / lat_long_df['totale_casi'].sum())*1000

lat_long_df['data'] = lat_long_df['data'].map(lambda item: pd.to_datetime(str(item)).strftime('%Y-%m-%d'))

parsed_data_datasource = []
for row in lat_long_df.iterrows():
    parsed_data_datasource.append([
        row[1]['lat']*random.uniform(1,1.001), 
        row[1]['long']*random.uniform(1,1.001), 
        row[1]['weight'], 
        row[1]['data']
    ])

data = []
time_indexes = []
for k, v in pd.DataFrame.from_records(parsed_data_datasource, columns=['lat', 'lon', 'weight', 'data']).groupby('data'):
    res = v[['lat', 'lon', 'weight']]
    data.append(res.values.tolist())
    time_indexes.append(pd.to_datetime(str(k)).strftime('%Y-%m-%d'))


m = folium.Map([48.0, 5.0], tiles="stamentoner", zoom_start=6)

hm = plugins.HeatMapWithTime(data, index=time_indexes)

hm.add_to(m)

len(data[0])
m

In [ ]:
an = pd.read_csv('COVID-19/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv')

an
an['tamponi/totale_casi'] = an['tamponi']/an['totale_casi']
an['ricoverati_con_sintomi/terapia_intensiva'] = an['terapia_intensiva']/an['ricoverati_con_sintomi'] * 100

In [ ]:
an['data'] = pd.to_datetime(an['data'])
an = an[['data', 'tamponi/totale_casi']]
an.set_index('data', inplace=True)

plot = an.plot(linewidth=2.0, grid=True, title="Numero tamponi vs totale infetti")
plot.set_xlabel("Data")
plot.set_ylabel("Tamponi/totale infetti (isolati o ospedalizzati)")
plot.legend(bbox_to_anchor=(1, 1))

fig = plot.get_figure()
fig.savefig("testsVsInfected-italia.png")

In [ ]:
an = pd.read_csv('COVID-19/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv')
an['tamponi/totale_casi'] = an['tamponi']/an['totale_casi']
an['ricoverati_con_sintomi/terapia_intensiva'] = an['terapia_intensiva']/an['ricoverati_con_sintomi'] * 100
an['data'] = pd.to_datetime(an['data'])
an = an[['data', 'ricoverati_con_sintomi/terapia_intensiva']]
an.set_index('data', inplace=True)

plot = an.plot(linewidth=2.0, grid=True, title="Percentuale Terapia Intensiva")
plot.set_xlabel("Data")
plot.set_ylabel("Percentuale di persone in Terapia intensiva vs ricoverati")
plot.legend(bbox_to_anchor=(1, 1))

fig = plot.get_figure()
fig.savefig("TI_hospitalized-italia.png")